In [2]:
%%capture
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/WhenPigsFlyContext/baselines/IVSN
! pip install ml-collections

In [3]:
import sys
import cv2
import time
from matplotlib import pyplot as plt
from tqdm import tqdm, trange
import numpy as np
import pandas as pd
import pickle
import random
import copy

import os
import shutil
from PIL import Image, ImageDraw

import torch
from torch.utils.data import Dataset
from torchvision.transforms.functional import to_tensor, normalize
from torchvision import transforms

from coco.dataset import COCODataset, COCODatasetRandom
from utils import *
sys.path.append("..")

In [5]:
%%capture
test_annotations = "../../debug/coco18_test.json"
test_imagedir = "../../coco18_test"
category_dic_dir = "../../coco18_idx_test612.pkl"
image_list = os.listdir(test_imagedir)
input_images = COCODatasetRandom(test_annotations, test_imagedir, image_size =(224,224), category_dic_dir = category_dic_dir, normalize_means=[0.485, 0.456, 0.406], normalize_stds=[0.229, 0.224, 0.225])

In [6]:
# define IVSN model
class IVSN(nn.Module):
  def __init__(self, model):
      super(IVSN, self).__init__()
      self.features = nn.Sequential(*list(model.children())[0][:30])
      for param in self.features.parameters():
        param.requires_grad_ = False

  def forward(self, x):
      x = self.features(x)
      return x

from torch.nn.modules.conv import Conv2d
ConvSize, NumTemplates, Mylayer = 1, 512, 31
TotalTrials, targetsize, stimulisize = 600, 32, 224
MMconv = Conv2d(NumTemplates, 1, kernel_size = (ConvSize, ConvSize), stride = (1, 1), padding = (1, 1))

In [7]:
model_vgg = models.vgg16(pretrained=True)
model_ivsn = IVSN(model_vgg)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [8]:
num_pics, size, image_size, shorter_side = len(input_images), 48, (320, 512), 128
IVSN_attention_map, scanpath = {}, {}
IVSN_res = list()

with torch.no_grad():
    for id in trange(num_pics):
        context_images, target_images, bbox, labels_cpu = input_images[id]
        # compute the ratio between width and height of the target, and set shorter side 128
        # aspect_ration = (bbox[2].item()*image_size[1])/(bbox[3].item()*image_size[0])
        tg_resize = (shorter_side, shorter_side)
        
        # get attention map from IVSN model
        context_images = transforms.Resize(image_size)(context_images)
        context_ivsn = context_images.view(1, 3, 320, 512)
        target_feedIVSN = transforms.Resize(tg_resize)(target_images)
        target_ivsn = target_feedIVSN.view(1, 3, tg_resize[0], tg_resize[1])
        cont_output_ivsn = model_ivsn(context_ivsn)
        tg_output_ivsn = model_ivsn(target_ivsn)
        MMconv.weight = torch.nn.Parameter(tg_output_ivsn)
        attention_IVSN = MMconv.forward(cont_output_ivsn)
        attention_IVSN = attention_IVSN.detach().squeeze(0)

        # # generate key for vit attention map
        label_name = input_images.idx2label[labels_cpu]

        tg_loc = bbox_cordinates(bbox, 512, 320)

        # process IVSN attention map
        mask_IVSN = transforms.Resize(image_size)(attention_IVSN)
        mask_IVSN = torch.divide(mask_IVSN, mask_IVSN.max())

        IVSN_attention_map[id] = copy.deepcopy(mask_IVSN)

        IVSN_num, path = searchProcesswithPath(tg_loc, mask_IVSN, image_size, size)
        scanpath[id] = path
        IVSN_res.append(IVSN_num)
        print('IVSN_' + label_name +  "_" + str(id) + ': ' + str(IVSN_num), end = '\t')


  0%|          | 1/612 [00:02<29:22,  2.88s/it]

IVSN_chair_0: 13	

  0%|          | 2/612 [00:05<28:39,  2.82s/it]

IVSN_chair_1: 2	

  0%|          | 3/612 [00:08<28:04,  2.77s/it]

...

IVSN_microwave_609: 16	

100%|█████████▉| 611/612 [19:21<00:01,  1.56s/it]

IVSN_microwave_610: 19	

100%|██████████| 612/612 [19:22<00:00,  1.90s/it]

IVSN_microwave_611: 2	

In [14]:
IVSN_res_fig5 = {}
IVSN_res_fig5['search_list'] = IVSN_res
IVSN_res_fig5['attention_map'] = IVSN_attention_map
IVSN_res_fig5['scanpath'] = scanpath

In [15]:
with open("../results/COCO/[Fig5]coco_IVSN_res.pkl", "wb") as tf:
    pickle.dump(IVSN_res_fig5, tf)

In [ ]:
IVSN_accu_OR = model_performance(IVSN_res, len(IVSN_res))
IVSN_accu_OR[:11]

[0,
 0.0,
 0.4722222222222222,
 0.5277777777777778,
 0.5735294117647058,
 0.6062091503267973,
 0.630718954248366,
 0.6584967320261438,
 0.6748366013071896,
 0.6911764705882353,
 0.7026143790849673]

In [ ]:
with open("../results/COCO/coco_IVSN_OR_accu_performance.pkl", "wb") as tf:
    pickle.dump(IVSN_accu_OR, tf)